In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("r/depression analysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/14 17:41:43 WARN Utils: Your hostname, cachyos, resolves to a loopback address: 127.0.1.1; using 192.168.0.128 instead (on interface wlan0)
25/12/14 17:41:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/14 17:41:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
submissions = spark.read.json("depression_submissions")
comments = spark.read.json("depression_comments")

submissions.show(5)
comments.show(5)

25/12/14 17:41:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-------------+-------------------+---------------+-----------+--------+-----------+--------------+------------------+-----------------------------+----------------------+---------------------+------------------------+-----------------+-----------------------+-----------------+---------------+-----------------+--------------------+--------------+--------+-------------+---------+----------+--------------+--------+------------+--------+-------+------------------+------------+-------+-----------+----------------+---------------------+---------------+-------------+---------------+-----+------+----+-------+---------+------+--------+------+----------+-----+----------------------+----------------+-------+-------------------+----------------------+------------------+-------+--------+-----+---------------------------+--------------------+-------------------+----------------------+---------------+---------------------+---------------+------+-----+--------------------+----------+--------+--

* Too many useless columns! 
* We pick what's important for us.

### Comments:

In [3]:
comments = comments.select("id", "author", "created_utc", "parent_id", "link_id", "score")
comments.show(10)

+----+-----------+-----------+---------+--------+-----+
|  id|     author|created_utc|parent_id| link_id|score|
+----+-----------+-----------+---------+--------+-----+
|7xa0|HappyRabbit| 1230776046| t3_7mqdw|t3_7mqdw|    1|
|7y9n|  [deleted]| 1230780294| t3_7mqo7|t3_7mqo7|    3|
|7zel|  [deleted]| 1230786613| t3_7mqur|t3_7mqur|    1|
|7zgf|     cluuxz| 1230786885| t3_7mqyx|t3_7mqyx|    5|
|7zgn|    cfabbro| 1230786916| t3_7mqur|t3_7mqur|    6|
|7zo9| Ostrich159| 1230788123| t3_7mqo7|t3_7mqo7|    6|
|7zxq|redmosquito| 1230789682|  t1_7zgf|t3_7mqyx|    3|
|7zyg|    simmias| 1230789820| t3_7mqur|t3_7mqur|    7|
|802k|      Sawta| 1230790554| t3_7mqyx|t3_7mqyx|    6|
|813r| Ostrich159| 1230796786|  t1_7zgf|t3_7mqyx|    4|
+----+-----------+-----------+---------+--------+-----+
only showing top 10 rows


We see that:
* direct comment to the post have the same link_id and parent_id starting with "t3_"
* replies to other comments have a parent_id starting with "t1_" and ends with the id of the parent comment
* we can use the parent_id prefix type and get red of the the columns link_id and score
* for embedded comments we know the author of the parent comment but for direct comments we dont know the submitter that they are interracted with
* we can join with the submissions to get that information and build our graph
* for the created_utc column we need to transform it into something human readable

In [4]:
# Deleting the link_id and score columns
comments = comments.select("id", "author", "created_utc", "parent_id")

# transforming the created_utc column
# first we cast the column into a long
comments = comments.withColumn("created_utc", F.col("created_utc").cast("double").cast("long"))

comments = comments.withColumn("year", F.year(F.from_unixtime("created_utc").cast("timestamp"))) \
       .withColumn("month", F.month(F.from_unixtime("created_utc").cast("timestamp"))) \
       .withColumn("day", F.dayofmonth(F.from_unixtime("created_utc").cast("timestamp")))

comments.orderBy(F.col("created_utc").desc()).show(10)

+-------+-----------------+-----------+----------+----+-----+---+
|     id|           author|created_utc| parent_id|year|month|day|
+-------+-----------------+-----------+----------+----+-----+---+
|m4rvixo|      fmylife2024| 1735689484|t1_m4rv20i|2025|    1|  1|
|m4rv3xo|     Unreal_catto| 1735689327|t3_1hqevuc|2025|    1|  1|
|m4rv20i|Fabulous_Owl_8471| 1735689307|t1_m4ruqba|2025|    1|  1|
|m4rv1ox|        [deleted]| 1735689304|t3_1frqlk0|2025|    1|  1|
|m4ruqba|      fmylife2024| 1735689184|t1_m4rrqqh|2025|    1|  1|
|m4ru0xd|     Nutsyblazzer| 1735688918|t1_m4r70da|2025|    1|  1|
|m4rtwm7| Single_Berry7546| 1735688872|t1_m4rpwi1|2025|    1|  1|
|m4rtuzw|Fabulous_Owl_8471| 1735688855|t3_1hqnkkr|2025|    1|  1|
|m4rtm0a| Single_Berry7546| 1735688761|t1_m4rm736|2025|    1|  1|
|m4rthad|   tarteframboise| 1735688712|t3_1hqqtay|2025|    1|  1|
+-------+-----------------+-----------+----------+----+-----+---+
only showing top 10 rows


### Submissions:

In [5]:
submissions = submissions.select("id", "author", "created_utc", "score", "num_comments")
submissions.show(10)

+-----+-----------+-----------+-----+------------+
|   id|     author|created_utc|score|num_comments|
+-----+-----------+-----------+-----+------------+
|7mqco|  [deleted]| 1230773002|    1|           0|
|7mqdw|HappyRabbit| 1230773772|    1|           1|
|7mqo7|  [deleted]| 1230780179|    7|           2|
|7mqur|  [deleted]| 1230784330|    6|           7|
|7mqyx|  [deleted]| 1230786778|    7|          11|
|7mr2a| Ostrich159| 1230788514|    4|           0|
|7mrbw|  [deleted]| 1230794035|    7|           1|
|7mrg3|  [deleted]| 1230796700|    9|           7|
|7mrp0| OctopusMan| 1230802690|   12|          14|
|7mru2|   i_love_u| 1230806086|    1|           0|
+-----+-----------+-----------+-----+------------+
only showing top 10 rows


* For the submissions we dont need the columns created_utc, score, num_comments
* the dataframe is needed only for joining to find the submission author

In [6]:
submissions = submissions.select("id", "author")
submissions.show(5)

+-----+-----------+
|   id|     author|
+-----+-----------+
|7mqco|  [deleted]|
|7mqdw|HappyRabbit|
|7mqo7|  [deleted]|
|7mqur|  [deleted]|
|7mqyx|  [deleted]|
+-----+-----------+
only showing top 5 rows


### Graph dataframe:

In [7]:
comments_to_posts = comments.filter(F.col("parent_id").startswith("t3_")) \
    .withColumn("post_id", F.regexp_replace(F.col("parent_id"), "^t3_", "")) \
    .join(submissions, F.col("post_id") == submissions["id"], "inner") \
    .select(
        comments["author"].alias("source"),
        submissions["author"].alias("target"),
        comments["created_utc"].alias("timestamp"),
        F.lit("comment_on_post").alias("interaction_type"),
        comments["year"],
        comments["month"],
        comments["day"]
    )

comments_to_posts.show(5)

+---------------+-------------+----------+----------------+----+-----+---+
|         source|       target| timestamp|interaction_type|year|month|day|
+---------------+-------------+----------+----------------+----+-----+---+
|FloralChester26|    [deleted]|1672517547| comment_on_post|2022|   12| 31|
| Pure_Goat_9428|    [deleted]|1672600359| comment_on_post|2023|    1|  1|
|      [deleted]|sarcasticicon|1672529728| comment_on_post|2023|    1|  1|
|      [deleted]|sarcasticicon|1672529742| comment_on_post|2023|    1|  1|
|   DedInside50s|   27Club2023|1672533471| comment_on_post|2023|    1|  1|
+---------------+-------------+----------+----------------+----+-----+---+
only showing top 5 rows


In [8]:
comment_replies = comments.filter(F.col("parent_id").startswith("t1_")) \
    .withColumn("parent_comment_id", F.regexp_replace(F.col("parent_id"), "^t1_", "")) \
    .alias("c1") \
    .join(comments.alias("c2"), F.col("c1.parent_comment_id") == F.col("c2.id"), "inner") \
    .select(
        F.col("c1.author").alias("source"),
        F.col("c2.author").alias("target"),
        F.col("c1.created_utc").alias("timestamp"),
        F.lit("comment_on_comment").alias("interaction_type"),
        F.col("c1.year"),
        F.col("c1.month"),
        F.col("c1.day")
    )

comment_replies.show(5)

+---------+---------------+----------+------------------+----+-----+---+
|   source|         target| timestamp|  interaction_type|year|month|day|
+---------+---------------+----------+------------------+----+-----+---+
|FokkeNews| IheartDaRegion|1231183607|comment_on_comment|2009|    1|  5|
|[deleted]| IheartDaRegion|1237915656|comment_on_comment|2009|    3| 24|
|[deleted]|      [deleted]|1239601618|comment_on_comment|2009|    4| 13|
|  MerEtAl|      [deleted]|1239646298|comment_on_comment|2009|    4| 13|
|[deleted]|anonymousreddit|1240244750|comment_on_comment|2009|    4| 20|
+---------+---------------+----------+------------------+----+-----+---+
only showing top 5 rows


In [ ]:
edges = comments_to_posts.union(comment_replies) \
    .filter((F.col("source") != "[deleted]") & (F.col("target") != "[deleted]") & (F.col("source") != F.col("target"))) \
    .orderBy("timestamp")

edges.show(10)

+-------------+-----------+----------+------------------+----+-----+---+
|       source|     target| timestamp|  interaction_type|year|month|day|
+-------------+-----------+----------+------------------+----+-----+---+
|  HappyRabbit|HappyRabbit|1230776046|   comment_on_post|2009|    1|  1|
|  redmosquito|     cluuxz|1230789682|comment_on_comment|2009|    1|  1|
|   Ostrich159|     cluuxz|1230796786|comment_on_comment|2009|    1|  1|
|     cujo3017|     cluuxz|1230800681|comment_on_comment|2009|    1|  1|
|   OctopusMan| OctopusMan|1230803293|   comment_on_post|2009|    1|  1|
|bobkingof12vs| OctopusMan|1230805754|comment_on_comment|2009|    1|  1|
|bobkingof12vs|   i_love_u|1230807596|   comment_on_post|2009|    1|  1|
|     g2petter| OctopusMan|1230812559|comment_on_comment|2009|    1|  1|
|   julenissen| OctopusMan|1230816912|comment_on_comment|2009|    1|  1|
|      lemming| OctopusMan|1230833115|comment_on_comment|2009|    1|  1|
+-------------+-----------+----------+-------------

In [ ]:
cmt_on_cmt_count = comment_replies.count()
cmt_on_pst_count = comments_to_posts.count()

total = cmt_on_cmt_count + cmt_on_pst_count
final = edges.count()
deleted = total - final
deleted_prct = (deleted/total)*100

stats = spark.createDataFrame([
    Row(total_count=total, 
        count_after_deletion=final, 
        deleted=deleted, 
        deleted_prct=round(deleted_prct,2))
])

stats.show()

+-----------+--------------------+-------+------------+
|total_count|count_after_deletion|deleted|deleted_prct|
+-----------+--------------------+-------+------------+
|    6377886|             4178188|2199698|       34.49|
+-----------+--------------------+-------+------------+

